In [2]:
import pandas as pd
from datetime import datetime, timedelta
import requests
import calendar

In [ ]:
print(datetime.today().timestamp())

In [ ]:
def date_to_unix_timestamp(date):
    return int(calendar.timegm(date.timetuple())) 


startdate = datetime(year= 2023, month= 10, day= 26, hour= 23, minute= 0, second= 0)
enddate = datetime(year= 2024, month= 10, day= 25, hour= 12, minute= 0, second= 0)

print(date_to_unix_timestamp(startdate))
print(date_to_unix_timestamp(enddate))

<h1 style = "color: yellow">Incremental Training (Online Learning)</h1>

## Historical Weather API

In [5]:
def convertUTCtoUnixForCurrent(date):
    return int(datetime.today().timestamp()) #

def date_to_unix_timestamp(date):
    return int(calendar.timegm(date.timetuple())) 

def getResponseFromHistoricalAPI(lat, lon, startDate, endDate, apiKey):
    endpoint = f"https://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=hour&start={startDate}&end={endDate}&appid={apiKey}"
    print(endpoint)

    try:
        response = requests.get(endpoint)
        if response.status_code == 200:
            print("Successful request")
            return response.json()
        
        else:
            print(f"Something wrong:\n\
                    Code: {response.status_code},\n\
                    Message: {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None
    
def transformResponseHistoricalToDataframe(lat = 21.0245, lon = 105.841, \
                                 startDate = datetime(year= 2024, month= 1, day= 1, hour= 0, minute= 0, second= 0), \
                                 endDate = datetime.today(), \
                                 apiKey = "89f98790ae72fd8a2da6032ba8e9c7b3") -> pd.DataFrame:
    listKeys = ["TimeStamp", "Temperature", "Feels Like", "Pressure", "Humidity", "Wind Speed", "Cloud", "Weather Description Main", "Rain (1h)"]
    dfDict = {f"{key}": [] for key in listKeys}

    # Initialize empty list:
    timestamp = []
    temperature = []
    feelslike = []
    pressure = []
    humidity = []
    windspeed = []
    cloud = []
    weatherdescriptionmain = []
    rain1h = []

    while startDate <= endDate:
        startTimestamp = startDate
        endTimestamp = startTimestamp + timedelta(weeks= 1)

        startTimestampUnix = date_to_unix_timestamp(date= startTimestamp)
        endTimestampUnix = date_to_unix_timestamp(date= endTimestamp)


        response = getResponseFromHistoricalAPI(lat= lat, lon= lon, startDate= startTimestampUnix, endDate= endTimestampUnix, apiKey= apiKey)

        for i in range(len(response["list"])):
            timestamp.append(datetime.utcfromtimestamp(response["list"][i]['dt']).strftime('%Y-%m-%d %H:%M:%S'))
            temperature.append(response["list"][i]["main"].get("temp"))
            feelslike.append(response["list"][i]["main"].get("feels_like"))
            pressure.append(response["list"][i]["main"].get("pressure"))
            humidity.append(response["list"][i]["main"].get("humidity"))
            windspeed.append(response["list"][i]["wind"].get("speed"))
            cloud.append(response["list"][i]["clouds"].get("all"))
            weatherdescriptionmain.append(response["list"][i]["weather"][0].get("main"))
            if response["list"][i].get("rain"):
                rain1h.append(response["list"][i]["rain"].get("1h", 0))
            else:
                rain1h.append(None)
        
        startDate += timedelta(weeks= 1, hours= 1)

    listValues = [timestamp, temperature, feelslike, pressure, humidity, windspeed, cloud, weatherdescriptionmain, rain1h]

    for i in range(9):
        dfDict[listKeys[i]] = listValues[i]

    df = pd.DataFrame(dfDict)
    df = df.applymap(lambda x: pd.to_numeric(x, errors= "ignore"))
    df["TimeStamp"] = pd.to_datetime(df["TimeStamp"])

    return df 

dfHistory = transformResponseHistoricalToDataframe()
dfHistory

https://history.openweathermap.org/data/2.5/history/city?lat=21.0245&lon=105.841&type=hour&start=1704067200&end=1704672000&appid=89f98790ae72fd8a2da6032ba8e9c7b3
Successful request
https://history.openweathermap.org/data/2.5/history/city?lat=21.0245&lon=105.841&type=hour&start=1704675600&end=1705280400&appid=89f98790ae72fd8a2da6032ba8e9c7b3


/tmp/ipykernel_9587/1525354289.py:55: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  timestamp.append(datetime.utcfromtimestamp(response["list"][i]['dt']).strftime('%Y-%m-%d %H:%M:%S'))


Successful request
https://history.openweathermap.org/data/2.5/history/city?lat=21.0245&lon=105.841&type=hour&start=1705284000&end=1705888800&appid=89f98790ae72fd8a2da6032ba8e9c7b3
Successful request
https://history.openweathermap.org/data/2.5/history/city?lat=21.0245&lon=105.841&type=hour&start=1705892400&end=1706497200&appid=89f98790ae72fd8a2da6032ba8e9c7b3
Successful request
https://history.openweathermap.org/data/2.5/history/city?lat=21.0245&lon=105.841&type=hour&start=1706500800&end=1707105600&appid=89f98790ae72fd8a2da6032ba8e9c7b3
Successful request
https://history.openweathermap.org/data/2.5/history/city?lat=21.0245&lon=105.841&type=hour&start=1707109200&end=1707714000&appid=89f98790ae72fd8a2da6032ba8e9c7b3
Successful request
https://history.openweathermap.org/data/2.5/history/city?lat=21.0245&lon=105.841&type=hour&start=1707717600&end=1708322400&appid=89f98790ae72fd8a2da6032ba8e9c7b3
Successful request
https://history.openweathermap.org/data/2.5/history/city?lat=21.0245&lon=105

/tmp/ipykernel_9587/1525354289.py:76: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: pd.to_numeric(x, errors= "ignore"))
/tmp/ipykernel_9587/1525354289.py:76: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.applymap(lambda x: pd.to_numeric(x, errors= "ignore"))


,TimeStamp,Temperature,Feels Like,Pressure,Humidity,Wind Speed,Cloud,Weather Description Main,Rain (1h)
0,2024-01-01 00:00:00,292.15,292.12,1017,77,1.77,74,Clouds,NaN
1,2024-01-01 01:00:00,293.15,293.14,1018,74,2.03,100,Clouds,NaN
2,2024-01-01 02:00:00,294.15,294.13,1019,70,2.24,100,Clouds,NaN
3,2024-01-01 03:00:00,297.15,297.36,1019,67,2.04,100,Clouds,NaN
4,2024-01-01 04:00:00,299.15,299.15,1018,63,2.01,100,Clouds,NaN
...,...,...,...,...,...,...,...,...,...
7160,2024-10-25 08:00:00,304.15,302.37,1007,23,3.78,100,Clouds,NaN
7161,2024-10-25 09:00:00,304.15,302.63,1007,27,3.24,100,Clouds,NaN
7162,2024-10-25 10:00:00,303.15,302.14,1007,33,2.25,100,Clouds,NaN
7163,2024-10-25 11:00:00,302.15,301.50,1008,37,1.80,100,Clouds,NaN


In [6]:
dfHistory.to_csv("history.csv")

In [8]:
dfHistory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7165 entries, 0 to 7164
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   TimeStamp                 7165 non-null   datetime64[ns]
 1   Temperature               7165 non-null   float64       
 2   Feels Like                7165 non-null   float64       
 3   Pressure                  7165 non-null   int64         
 4   Humidity                  7165 non-null   int64         
 5   Wind Speed                7165 non-null   float64       
 6   Cloud                     7165 non-null   int64         
 7   Weather Description Main  7165 non-null   object        
 8   Rain (1h)                 1302 non-null   float64       
dtypes: datetime64[ns](1), float64(4), int64(3), object(1)
memory usage: 503.9+ KB


## Current Weather API

In [ ]:
def getResponseFromCurrentAPI(lat = 21.0245, lon =105.841, apiKey = "89f98790ae72fd8a2da6032ba8e9c7b3"):
    endpoint = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={apiKey}"
    print(endpoint)

    try:
        response = requests.get(endpoint)
        if response.status_code == 200:
            print("Successful request")
            return response.json()
        
        else:
            print(f"Something wrong:\n\
                    Code: {response.status_code},\n\
                    Message: {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

def transformResponseCurrentToDataframe(lat = 21.0245, lon =105.841, apiKey = "89f98790ae72fd8a2da6032ba8e9c7b3") -> pd.DataFrame:
    response = getResponseFromCurrentAPI(lat= lat, lon= lon, apiKey= apiKey)
    listKeys = ["Country", "Name", "TimeStamp", "Temperature", "Feels Like", "Pressure", "Humidity", "Wind Speed", "Cloud", "Weather Description Main", "Rain (1h)"]
    dfDict = {f"{key}": None for key in listKeys}

    # Extract needed atribute from Response(JSON)
    country = response["sys"].get("country")
    name = response["name"]
    timestamp = datetime.utcfromtimestamp(response['dt']).strftime('%Y-%m-%d %H:%M:%S')
    temperature = response["main"].get("temp")
    feelslike = response["main"].get("feels_like")
    pressure = response["main"].get("pressure")
    humidity = response["main"].get("humidity")
    windspeed = response["wind"].get("speed")
    cloud = response["clouds"].get("all")
    weatherdescriptionmain = response["weather"][0].get("main")
    if response.get("rain"):
        rain1h = response["rain"].get("1h", 0)
    else:
        rain1h = None

    listValues = [country, name, timestamp, temperature, feelslike, pressure, humidity, windspeed, cloud, weatherdescriptionmain, rain1h]

    for i in range(11):
        dfDict[listKeys[i]] = listValues[i]

    df = pd.DataFrame(dfDict, index= [0]) # index = [0]: ensure that Pandas treat each value in dfDict as single row
    df = df.applymap(lambda x: pd.to_numeric(x, errors= "ignore"))
    df["TimeStamp"] = pd.to_datetime(df["TimeStamp"])

    return df 

    
lat = 21.0245
lon = 105.841
apiKey = "89f98790ae72fd8a2da6032ba8e9c7b3"

dfCurrent = transformResponseCurrentToDataframe(lat= lat, lon= lon, apiKey= apiKey)
dfCurrent

In [ ]:
dfCurrent.info()